# Pre

In [14]:
import numpy as np
import mudata as mu
import scipy.sparse as sp
import loompy as lp

In [15]:
def write_loom(adata, filename, layer=None, use_raw=False):
    """Write AnnData object as loom that is compatible with SCENIC

    Parameters
    ----------
    adata : AnnData
        The annotated data matrix of shape `n_obs` x `n_vars`.
        Rows correspond to cells and columns to genes.
    filename : str
        Filename to save to
    layer : str
        Layer to save instead of `X`. If `None`, `X` is saved.

    Returns
    -------
    None
    """
    print(f"Saving as loom to {filename}")
    if use_raw:
        adata = adata.raw.to_adata()  #only if adata has RAW saved and thats what you want!!
    row_attrs = dict(zip(adata.var.reset_index().columns, adata.var.reset_index().values.T))
    col_attrs = dict(zip(adata.obs.reset_index().columns, adata.obs.reset_index().values.T))
    row_attrs["Gene"] = np.array(adata.var_names)
    col_attrs["CellID"] = np.array(adata.obs_names)
    if layer is not None:
        X = adata.layers[layer]
    else:
        X = adata.X
    if sp.issparse(X):
        X = X.toarray()
    col_attrs["nGene"] = np.array(np.sum(X.transpose() > 0, axis=0)).flatten()
    col_attrs["nUMI"] = np.array(np.sum(X.transpose(), axis=0)).flatten()
    lp.create(filename, X.transpose(), row_attrs, col_attrs)

In [16]:
path_data = "/cellar/users/aklie/data/datasets/neurips2021_small/annotation/2023_12_05/mdata.h5mu"
path_out = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic/rna.loom"

In [17]:
mdata = mu.read_h5mu(path_data)
mdata

/cellar/users/aklie/opt/miniconda3/envs/test_scenicplus_dev/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_scenicplus_dev/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


MuData object with n_obs × n_vars = 123 × 8704
  obs:	'celltype', 'batch', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
  2 modalities
    atac:	123 x 8192
      obs:	'pseudotime_order'
      obsm:	'X_lsi', 'X_umap'
      layers:	'counts'
    rna:	123 x 512
      obs:	'pseudotime_order'
      var:	'gene_id'
      obsm:	'X_pca', 'X_umap'
      layers:	'counts'

In [18]:
write_loom(
    mdata.mod["rna"],
    filename=path_out,
    layer="counts"
)

Saving as loom to /cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic/rna.loom
